In [6]:
import pandas as pd

In [4]:
df_before = pd.read_excel("/content/drive/MyDrive/each_sum_new.xlsx")
df_before.info()

# 2025-04-02-00:00 -> 2025-04-02 00:00으로 변환
for i in range(len(df_before)):
  df_before.at[i, '매출일'] = df_before.at[i, '매출일'][:10] + " " + df_before.at[i, '매출일'][11:]

# 종료시간 생성(결제시간 + 기본 70분 + 추가 결제액 100원당 10분 추가 = 종료시간 임을 이용) 후 df 생성하는 과저  -> 종료시간 생성 식이 맞는지 점검 필요(기존 식 or 결제시간에 70분 더하는건 빼는 방시)
df_end_time = list()
df_pay_time = list()
for i in range(len(df_before)):
  if not(df_before.at[i, '이용액'] == 0):
    df_end_time.append(pd.to_datetime(df_before.at[i, '매출일']) + pd.Timedelta(minutes=0.1*df_before.at[i, '이용액']))
    df_pay_time.append(pd.to_datetime(df_before.at[i, '매출일']))
df_end_time = pd.DataFrame(df_end_time, columns = ['종료시간'])
df_pay_time = pd.DataFrame(df_pay_time, columns = ['결제시간'])
df = pd.concat([df_end_time,df_pay_time], axis=1)

# df 통해 time dataframe 생성하는 과정
time = [[],[]]
time[0].append(pd.to_datetime(df.at[0, '결제시간']))
time[1].append(0)
index = 0
# "현재 시간이 마지막 시간보다 작거나 같으면..." (계속 5분씩 더해라)  -> df.at에 pd.to_datetime을 씌우는 것이 맞는지 점검 필ㅇ
while (time[0][index] <= pd.to_datetime(df['종료시간'].max())):
  time[0].append(time[0][index] + pd.Timedelta(minutes=5))
  time[1].append(0)
  index += 1
data = {
    '결제시간': time[0],
    '세탁기사용량': time[1]
}
time= pd.DataFrame(data)

# 데이터 전처리(time dataframe 값(세탁기사용량 값)df dataframe 통해 채우는 과정)
i = 0
while i < len(df):
    j = 0
    while j < len(time):
        if df.at[i, '결제시간'] <= time.at[j, '결제시간'] < df.at[i, '종료시간']:
            time.at[j, '세탁기사용량'] += 1
        j += 1

    i += 1

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3948 entries, 0 to 3947
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   이용구분    3948 non-null   object
 1   수량      3948 non-null   int64 
 2   입금액     3948 non-null   int64 
 3   매출일     3948 non-null   object
 4   이용액     3948 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 154.3+ KB


In [7]:
time.head()

,결제시간,세탁기사용량
0,2025-04-01 00:00:00,1
1,2025-04-01 00:05:00,2
2,2025-04-01 00:10:00,2
3,2025-04-01 00:15:00,1
4,2025-04-01 00:20:00,2


In [8]:
time.to_excel(
    excel_writer = '/content/drive/MyDrive/time.xlsx',
    sheet_name = 'Sheet1',
    index = False
)